# Alineamientos

Funcion para crear el dotplot.
(DUDA: EN EL GRAFICO DE LA CLASE SE VEÍA DIFERENTE CON UN EJEMPLO... CONSULTAR)

In [1]:
import numpy as np

def make_dotplot(seq1,seq2):
    dotplot = np.zeros((len(seq1),len(seq2)))
    for i in range(len(seq1)):
        for j in range(len(seq2)):
            if seq1[i]==seq2[j]:
                dotplot[i,j]=1
    return dotplot

Funcion para encontrar el mejor alineamiento posible... (una especie de greedy, hecho rapido, tiene muchos 'huecos')

In [2]:
def obtain_best_alignment(dotplot):
    complete = False
    alignment_list = []
    i = 0
    j = 0
    while not complete:
        if i >= len(dotplot) or j >= len(dotplot[0]):
            complete = True
        else:
            alignment_list.append([i, j])
            if i + 1 < len(dotplot) and j + 1 < len(dotplot[0]) and dotplot[i + 1, j + 1] == 1:
                i += 1
                j += 1
            else:
                j += 1
    return alignment_list


Traduccion de la lista de alineamientos a una sintaxis mas amigable

In [9]:
def traduce_alignment(alignment_list, seq1, seq2):
    traduced_seq = ''
    for i in range(len(alignment_list)):
        if seq1[alignment_list[i][0]] == seq2[alignment_list[i][1]]:
                traduced_seq += f"{seq1[alignment_list[i][0]]}{i}"
        else:
            # traduced_seq += f"{seq1[alignment_list[i-1][0]]}"
            # Si se prefiere un guion en vez de repetir la ultima base:
            traduced_seq += "-"
    return traduced_seq
    

Funcion de alineamiento de secuencias

In [10]:
def alignment(seq1, seq2):
    dotplot = make_dotplot(seq1,seq2)
    best_alignment = obtain_best_alignment(dotplot)
    traduced_alignment = traduce_alignment(best_alignment, seq1, seq2)
    return traduced_alignment

seq1 = 'TACG'
seq2 = 'TATCG'
align = alignment(seq1, seq2)

print(align)

T0A1-C3G4


Función de alineamiento de biopython

In [11]:
from Bio import pairwise2

# Realizar el alineamiento
alignments = pairwise2.align.globalxx(seq1, seq2)

# Imprimir los alineamientos
for alignment in alignments:
    print(pairwise2.format_alignment(*alignment))

TA-CG
|| ||
TATCG
  Score=4

